In [ ]:
from jax import config
config.update("jax_debug_nans", True)
config.update("jax_enable_x64", True)
config.update("jax_disable_jit", False)
config.update("jax_log_compiles", False)

from pathlib import Path

from matplotlib import pyplot as plt
import numpy as np

from viperleed_jax.lib_tensors import read_tensor
from viperleed_jax.lib_phaseshifts import *
from viperleed_jax.data_structures import ReferenceData
from viperleed_jax.tensor_calculator import TensorLEEDCalculator

%matplotlib inline

In [ ]:
jax.devices()

In [ ]:
use_installable = True
if use_installable:
    import viperleed
    from viperleed.calc.files import poscar
    from viperleed.calc.files import parameters
    from viperleed.calc.classes.rparams import Rparams
    from viperleed.calc.files.beams import readIVBEAMS
    from viperleed.calc.files.phaseshifts import readPHASESHIFTS
    from viperleed.calc.files.vibrocc import readVIBROCC
else:
    # master
    import sys
    sys.path.append('/Users/alexander/GitHub/')
    import viperleed
    from viperleed.tleedmlib.files import poscar
    from viperleed.tleedmlib.files import parameters
    from viperleed.tleedmlib.classes.rparams import Rparams
    from viperleed.tleedmlib.files.beams import readIVBEAMS
    from viperleed.tleedmlib.files.phaseshifts import readPHASESHIFTS
    from viperleed.tleedmlib.files.vibrocc import readVIBROCC

In [ ]:
data_path = Path('tests') / 'test_data' / 'Cu_111'

In [ ]:
# Read in data from POSCAR and PARAMETERS files
slab = poscar.read(data_path / 'POSCAR')
rparams = parameters.read(data_path / 'PARAMETERS')
parameters.interpret(rparams, slab, silent=False)
slab.full_update(rparams)

# reading IVBEAMS
rparams.ivbeams = readIVBEAMS(data_path / 'IVBEAMS')
beam_indices = np.array([beam.hk for beam in rparams.ivbeams])

# reading VIBROCC
readVIBROCC(rparams, slab, data_path / 'VIBROCC')

# incidence angles
rparams.THETA = 0.0
rparams.PHI = 0.0

In [ ]:
LMAX = rparams.LMAX.max

In [ ]:
param_energies = np.linspace(rparams.THEO_ENERGIES.start,
                           rparams.THEO_ENERGIES.stop,
                           rparams.THEO_ENERGIES.n_energies)

## Read Tensor files

In [ ]:
read_tensor_num = lambda num: read_tensor(data_path / 'Tensors' / f'T_{num}', n_beams=len(rparams.ivbeams), n_energies=param_energies.size, l_max=LMAX+1)
non_bulk_atoms = [at for at in slab.atlist if not at.is_bulk]
tensors = [read_tensor_num(at.num) for at in non_bulk_atoms]

ref = ReferenceData(tensors, fix_lmax=10)

#delete tensors to free up memory
for t in tensors:
    del t
del tensors

In [ ]:
# read phase shifts
phaseshifts_path = data_path /  'PHASESHIFTS'
_, raw_phaseshifts, _, _ = readPHASESHIFTS(slab, rparams, readfile=phaseshifts_path,
                                       check=True, ignoreEnRange=False)


In [ ]:
# interpolate & assign phase shifts to atoms
site_indices = [np.argmax([at.site.isEquivalent(s) for s in slab.sitelist])
                for at in non_bulk_atoms]

# TODO: with this current implementation, we can not treat chemical
#       pertubations, nor vacancies. We need to implement this.
#       See e.g. iodeltas.generateDeltaInput()
#       (Treating vacancies requires setting zeros for that site)

phaseshifts = Phaseshifts(raw_phaseshifts, ref.energies, LMAX, site_indices)

# Calculator

In [ ]:
beam_indices = jnp.array([[1., 0.],
       [0., 1.],
       [1., 1.],
       [2., 0.],
       [0., 2.],
       [2., 1.],
       [1., 2.],
       [3., 0.],
       [0., 3.]])

In [ ]:
calculator = TensorLEEDCalculator(ref, phaseshifts, slab, rparams, beam_indices=beam_indices)

In [ ]:
centered_vib_amps = calculator.ref_vibrational_amps
centered_displacements = np.array([[0.0, 0.0, 0.0],]*5)

In [ ]:
%time ref_int = calculator.intensity(centered_vib_amps, centered_displacements)

In [ ]:
# set reference point
calculator.set_experiment_intensity(ref_int, param_energies)

# Intensity

In [ ]:
# some displacements to play with
spaced_displacements = [
    np.array([[i*0.01-0.05, 0.0, 0.0],] +[[0.00, 0.0, 0.0],]*4)
    for i in range (11)
]

In [ ]:
beam = 0

In [ ]:
%matplotlib inline
plt.figure()
for d in spaced_displacements:
    plt.plot(param_energies,
             calculator.intensity(centered_vib_amps, d)[:, beam])
plt.show()

In [ ]:
i = calculator.unperturbed_intensity
plt.plot(param_energies, i[:, 1])

In [ ]:
from viperleed_jax.constants import *

In [ ]:
ref_data = ref
EPS = 1e-8
vib_amps = jnp.asarray(centered_vib_amps)
displacements = jnp.asarray(centered_displacements)

energies = ref_data.energies
v_imag = ref_data.v0i

# energy dependent quantities
out_k_par2 = ref_data.kx_in
out_k_par3 = ref_data.ky_in

k_inside = jnp.sqrt(2*energies-2j*v_imag+1j*EPS)

# Propagator evaluated relative to the muffin tin zero i.e.
# it uses energy = incident electron energy + inner potential
out_k_par = out_k_par2**2 + out_k_par3**2
out_k_perp_inside = jnp.sqrt(
    ((2*energies-2j*v_imag)[:, jnp.newaxis] - out_k_par)
    + 1j*EPS
)

# Prefactors from Equation (41) from Rous, Pendry 1989
d_prefactors = jnp.einsum('e,eb,->eb',
    1/k_inside,
    1/out_k_perp_inside,
    1/(2*(calculator.unit_cell_area/BOHR**2)))

In [ ]:
plt.semilogy(
    param_energies,
    d_prefactors*prefactor, label=beam_indices)
plt.legend()

In [ ]:
beam_indices @ calculator.reciprocal_unit_cell.T

In [ ]:
theta, phi = .0, .0
from viperleed_jax.constants import BOHR

In [ ]:
cell = slab.ab_cell / BOHR
area = jnp.linalg.norm(jnp.cross(slab.ab_cell[0], slab.ab_cell[1]))/BOHR**2

2*jnp.pi*jnp.linalg.inv(slab.ab_cell / BOHR).T @ jnp.array([1., 0.])

In [ ]:
rec_unit_cell = 2*jnp.pi*jnp.linalg.inv(slab.ab_cell /BOHR).T

ref_data = ref
e_kin = ref_data.energies
v_real = ref_data.v0r
v_imag = ref_data.v0i
n_energies = e_kin.shape[0]
n_beams = beam_indices.shape[0]
# incident wave vector
in_k_vacuum = jnp.sqrt(jnp.maximum(0, 2 * (e_kin - v_real)))
in_k_par = in_k_vacuum * jnp.sin(theta)  # parallel component
in_k_par_2 = in_k_par * jnp.cos(phi)  # shape =( n_energy )
in_k_par_3 = in_k_par * jnp.sin(phi)  # shape =( n_energy )
in_k_perp_vacuum = jnp.empty_like(e_kin, dtype="complex64")
in_k_perp_vacuum = 2 * e_kin - in_k_par_2 ** 2 - in_k_par_3 ** 2 - 2 * 1j * v_imag
in_k_perp_vacuum = jnp.sqrt(in_k_perp_vacuum)

# outgoing wave vector components
in_k_par_components = jnp.stack((in_k_par_2, in_k_par_3))  # shape =(n_en, 2)
in_k_par_components = jnp.outer(in_k_par_components, jnp.ones(shape=(n_beams,))).reshape(
(n_energies, 2, n_beams))  # shape =(n_en ,2 ,n_beams)
out_wave_vec = jnp.dot(beam_indices, rec_unit_cell.T)  # shape =(n_beams, 2)
out_wave_vec = jnp.outer(jnp.ones_like(e_kin), out_wave_vec.transpose()).reshape((n_energies, 2, n_beams))  # shape =(n_en , n_beams)
out_k_par_components = in_k_par_components + out_wave_vec

# out k vector
out_k_perp_vacuum = (2*jnp.outer(e_kin-v_real,jnp.ones(shape=(n_beams,)))
            - out_k_par_components[:, 0, :] ** 2
            - out_k_par_components[:, 1, :] ** 2).astype(dtype="complex64")
out_k_perp = jnp.sqrt(out_k_perp_vacuum + 2*jnp.outer(v_real-1j*v_imag, jnp.ones(shape=(n_beams,))))
out_k_perp_vacuum = jnp.sqrt(out_k_perp_vacuum)

In [ ]:
a = out_k_perp_vacuum
c = in_k_vacuum * jnp.cos(theta)


prefactor = abs(jnp.exp(-1j * 0.1 * (jnp.outer(in_k_perp_vacuum, jnp.ones(shape=(n_beams,))) + out_k_perp
                                            ))) ** 2 * a.real / jnp.outer(c, jnp.ones(shape=(n_beams,))).real
plt.plot(param_energies, prefactor[:, :])


In [ ]:
param_energies

In [ ]:
1/slab.ab_cell*BOHR

# Interpolation

In [ ]:
plt.figure()
for d in spaced_displacements:
    plt.plot(calculator.interpolated(centered_vib_amps, d, deriv_deg=0)[:, beam])
plt.title("Interpolated Intensity")

In [ ]:
raise

In [ ]:
plt.figure()
for d in spaced_displacements:
    plt.plot(calculator.interpolated(centered_vib_amps, d, deriv_deg=1)[:, beam])
plt.title("Interpolated Derivative")

In [ ]:
from viperleed_jax.rfactor import pendry_y
plt.figure()
for d in spaced_displacements:
    intensity = calculator.interpolated(centered_vib_amps, d, deriv_deg=0)[:, beam]
    deriv = calculator.interpolated(centered_vib_amps, d, deriv_deg=1)[:, beam]
    plt.plot(pendry_y(intensity, deriv, 4.5))
plt.title("Interpolated Y-function")

# Rfactor

In [ ]:
# compile time
%time calculator.R_pendry_val_and_grad(centered_vib_amps, centered_displacements)

In [ ]:
# compile time
%time calculator.R_pendry_val_and_grad(centered_vib_amps, centered_displacements)

In [ ]:
R_arr = []
R_grad_arr = []
z_arr = []
for d in spaced_displacements:
    R, gradient = calculator.R_pendry_val_and_grad(centered_vib_amps, d)
    R_arr.append(R)
    R_grad_arr.append(gradient)
    z_arr.append(d[0][0])

In [ ]:
plt.figure()
plt.plot(z_arr, R_arr)

In [ ]:
plt.figure()
plt.plot(z_arr, [g[0,0] for g in R_grad_arr])
plt.plot(z_arr, [g[0,1] for g in R_grad_arr])

In [ ]:
plt.figure()
plt.plot(z_arr, [g[0,0] for g in R_grad_arr])
plt.plot(z_arr[:-1], jnp.diff(np.array(R_arr))/ (z_arr[1]-z_arr[0]))

# R2 - TODO

In [ ]:
ref_intensity_all_beams = delta_intensity(jnp.array([[0.0, 0.0, 0.0],]))

In [ ]:
lam_r2 = lambda z: jnp.real(((delta_intensity(jnp.array([[z, 0.0, 0.0],])) - ref_intensity_all_beams)**2).sum())

In [ ]:
z_arr = jnp.linspace(-0.05, 0.05, 100)
R2_arr = [lam_r2(r) for r in z_arr]
R2_grad_arr = [jax.grad(lam_r2)(r) for r in z_arr]

In [ ]:
plt.figure()
plt.plot(z_arr, R2_arr)

In [ ]:
plt.figure()
plt.plot(z_arr, R2_grad_arr)